In [1]:
import os

In [2]:
os.chdir('../')
os.getcwd()

'/home/zakaria/workspace/ai-photo-enhancer'

In [3]:
from src.envs.new_edit_photo import PhotoEditor
from src.envs.photo_env import PhotoEnhancementEnvTest
from src.sac.sac_inference import InferenceAgent
import yaml
import torch
from tensordict import TensorDict
import torchvision.transforms.v2.functional as F
from src.envs.env_dataloader import create_dataloaders
from torchmetrics.image import StructuralSimilarityIndexMeasure
from tqdm import tqdm
import numpy as np

In [4]:
model_path = 'experiments/runs/ResHist__224_128_aug_hist__2024-07-26_16-17-35'
model_path = 'experiments/runs/ResNet_10_sliders__224_128_aug__2024-07-23_21-23-35'
model_paths = ['experiments/runs/ResNet_10_sliders__224_128_aug__2024-07-23_21-23-35',
               'experiments/runs/ResHist__224_128_aug_hist__2024-07-26_16-17-35']
DEVICE='cpu'

In [5]:
class Config(object):
    def __init__(self, dictionary):
        self.__dict__.update(dictionary)

def load_preprocessor_agent(preprocessor_agent_path, device):
    # with open(
    #     os.path.join(preprocessor_agent_path, "configs/sac_config.yaml")
    # ) as f:
    #     sac_config_dict = yaml.load(f, Loader=yaml.FullLoader)
    with open(
        os.path.join(preprocessor_agent_path, "configs/env_config.yaml")
    ) as f:
        env_config_dict = yaml.load(f, Loader=yaml.FullLoader)
    with open(os.path.join("src/configs/inference_config.yaml")) as f:
        inf_config_dict = yaml.load(f, Loader=yaml.FullLoader)

    inference_config = Config(inf_config_dict)
    # sac_config = Config(sac_config_dict)
    env_config = Config(env_config_dict)

    inference_env = PhotoEnhancementEnvTest(
        batch_size=env_config.train_batch_size,
        imsize=env_config.imsize,
        training_mode=False,
        done_threshold=env_config.threshold_psnr,
        edit_sliders=env_config.sliders_to_use,
        features_size=env_config.features_size,
        discretize=env_config.discretize,
        discretize_step=env_config.discretize_step,
        use_txt_features=(
            env_config.use_txt_features
            if hasattr(env_config, "use_txt_features")
            else False
        ),
        augment_data=False,
        pre_encoding_device=device,
        pre_load_images=False,
        logger=None,
    )

    inference_config.device = device
    preprocessor_agent = InferenceAgent(inference_env, inference_config)
    preprocessor_agent.device = device
    preprocessor_agent.load_backbone(
        os.path.join(preprocessor_agent_path, "models", "backbone.pth")
    )
    preprocessor_agent.load_actor_weights(
        os.path.join(preprocessor_agent_path, "models", "actor_head.pth")
    )
    preprocessor_agent.load_critics_weights(
        os.path.join(preprocessor_agent_path, "models", "qf1_head.pth"),
        os.path.join(preprocessor_agent_path, "models", "qf2_head.pth"),
    )
    return preprocessor_agent,env_config

def generate_parameters(enhancer_agent,image, deterministic=True,DEVICE=DEVICE):
    input_image = image.to(DEVICE)
    input_image = input_image.permute(0, 3, 1, 2)
    IMAGE_SIZE = enhancer_agent.env.imsize
    input_image = F.resize(
        input_image,
        (IMAGE_SIZE, IMAGE_SIZE),
        interpolation=F.InterpolationMode.BICUBIC,
    )
    batch_observation = TensorDict(
        {
            "batch_images": input_image,
        },
        batch_size=[input_image.shape[0]],
    )
    parameters = enhancer_agent.act(
        batch_observation, deterministic=deterministic, n_samples=0
    )
    return parameters

In [6]:
def compute_500px(agent,env_config):
    photo_editor = PhotoEditor(env_config.sliders_to_use)
    ssim_metric = StructuralSimilarityIndexMeasure().to(DEVICE)
    test_512 = create_dataloaders(
        batch_size=1,
        image_size=env_config.imsize,
        use_txt_features=False,
        train=False,
        augment_data=False,
        shuffle=False,
        resize=False,
        pre_encoding_device=DEVICE,
        pre_load_images=False,
    )
    test_resized = create_dataloaders(
        batch_size=500,
        image_size=env_config.imsize,
        use_txt_features=(
            env_config.use_txt_features
            if hasattr(env_config, "use_txt_features")
            else False
        ),
        train=False,
        augment_data=False,
        shuffle=False,
        resize=True,
        pre_encoding_device=DEVICE,
        pre_load_images=True,
    )
    PSNRS = []
    SSIM = []
    agent.env.dataloader = test_resized
    agent.env.iter_dataloader = iter(test_resized)
    agent.env.batch_size = 500
    batch_images = agent.env.reset()
    parameters = agent.act(
        batch_images, deterministic=True
    )
    parameter_counter = 0
    for i, t in tqdm(test_512, position=0, leave=True):
        source = i / 255.0
        target = t / 255.0
        enhanced_image = source.permute(0, 2, 3, 1)
        enhanced_image = photo_editor(
            enhanced_image.to(DEVICE),
            parameters[parameter_counter].unsqueeze(0).to(DEVICE),
        )
        enhanced_image = enhanced_image.permute(0, 3, 1, 2)  # B,C,H,W
        psnr = (
            agent.env.compute_rewards(
                enhanced_image.to(DEVICE), target.to(DEVICE)
            ).item()
            + 50
        )
        ssim = ssim_metric(
            enhanced_image.to(DEVICE), target.to(DEVICE)
        ).item()
        PSNRS.append(psnr)
        SSIM.append(ssim)
        parameter_counter += 1
    mean_PSNRS = round(np.mean(PSNRS), 2)
    mean_SSIM = round(np.mean(SSIM), 3)
    return mean_PSNRS, mean_SSIM

In [7]:
agent,env_config = load_preprocessor_agent(model_path, DEVICE)
a,b=compute_500px(agent,env_config)

Preloading images and features...


100%|██████████| 500/500 [00:01<00:00, 264.42it/s]


Images and features preloaded and stored in GPU memory.


100%|██████████| 500/500 [00:47<00:00, 10.46it/s]


In [10]:
a


22.56